In [2]:
#Installing important libraries

In [3]:
!pip install torch_geometric 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 8.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=de22df27fb91f10692808fb89e59ae58f359876140bcf46a2fab693abe72f20c
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [4]:
import time

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
from torch_geometric.datasets import Planetoid
from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
#Defining the model

In [9]:
class GCNRW(nn.Module):
    def __init__(self, n_features, n_classes, n_hidden=6000, lambda_=10):
        super(GCNRW, self).__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.n_hidden = n_hidden
        self.lambda_ = lambda_
        self.W = nn.Parameter(torch.Tensor(n_features, n_hidden).uniform_(-1, 1))

    def forward(self, A, X, Y):
        A_hat = A + torch.eye(A.size(0)).to(A.device)
        A_hat2 = torch.mm(A_hat, A_hat)
        H = torch.sigmoid(-torch.mm(torch.mm(A_hat2, X), self.W))

        self.b = None

        if Y.shape[1] >= self.n_hidden:
            self.b = torch.inverse(H.t() @ H + self.lambda_*torch.eye(self.n_hidden)) @ H.t() @ Y
        else:
            self.b = H.t() @ torch.inverse(H @ H.t() + self.lambda_*torch.eye(H.shape[0])) @ Y

        Z = torch.mm(H, self.b)
        return Z

    def fit(self, A, X, Y):
        A = A.to(X.device)
        X = X.to(A.device)
        Y = Y.to(A.device)
        self.b = None  # Initialize output weights
        Z = self.forward(A, X,Y)  

        return Z

    def predict(self, A, X):
        A = A.to(X.device)
        X = X.to(A.device)
        A_hat = A + torch.eye(A.size(0)).to(A.device)
        A_hat2 = torch.mm(A_hat, A_hat)
        H = torch.sigmoid(-torch.mm(torch.mm(A_hat2, X), self.W))
        Z = torch.mm(H, self.b)
        return Z

In [10]:
# Load data

In [11]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
A = data.edge_index
X = data.x
Y = data.y
n_classes = dataset.num_classes

Processing...
Done!


In [12]:
# Convert target to one-hot

In [13]:
Y_onehot = F.one_hot(Y, num_classes=n_classes).float()

In [14]:
# Get adjacency matrix

In [15]:
num_nodes = data.num_nodes
A = torch.zeros(num_nodes, num_nodes)
A[data.edge_index[0], data.edge_index[1]] = 1


In [16]:
# Split data into training and test sets

In [17]:
train_mask, test_mask = train_test_split(range(num_nodes), test_size=0.8, stratify=Y)

A_train = A[train_mask][:, train_mask]
X_train = X[train_mask]
Y_train = Y_onehot[train_mask]
A_test = A[test_mask][:, test_mask]
X_test = X[test_mask]
Y_test = Y_onehot[test_mask]




In [18]:
# Create model

In [19]:
model = GCNRW(n_features=X.size(1), n_classes=n_classes,  n_hidden=8000, lambda_=15)


In [20]:
# Train on training set

In [21]:
start = time.time()
Z_train = model.fit(A_train, X_train, Y_train)

In [22]:
# Evaluate on train set

In [23]:
Z_train = model.predict(A_train, X_train)
train_acc = accuracy_score(Y_train.argmax(1), Z_train.argmax(1))
print("train time",time.time() - start)
print(f'Train accuracy: {train_acc:.4f}')




train time 0.5529205799102783
Train accuracy: 0.9815


In [24]:
# Evaluate on test set

In [25]:
start = time.time()
Z_test = model.predict(A_test, X_test)
test_acc = accuracy_score(Y_test.argmax(1), Z_test.argmax(1))
print("test time",time.time() - start)


print(f'Test accuracy: {test_acc:.4f}')

test time 0.8753726482391357
Test accuracy: 0.7933


In [26]:
#Employing the code on Citeseer data

In [27]:
# Load data

In [28]:
dataset = Planetoid(root='/tmp/Citeseer', name='Citeseer')
data = dataset[0]
A = data.edge_index
X = data.x
Y = data.y
n_classes = dataset.num_classes

Processing...
Done!


In [29]:
# Convert target to one-hot

In [30]:
Y_onehot = F.one_hot(Y, num_classes=n_classes).float()

In [31]:
# Get adjacency matrix

In [32]:
num_nodes = data.num_nodes
A = torch.zeros(num_nodes, num_nodes)
A[data.edge_index[0], data.edge_index[1]] = 1

In [33]:
# Split data into training and test sets


In [34]:
train_mask, test_mask = train_test_split(range(num_nodes), test_size=0.85, stratify=Y)

A_train = A[train_mask][:, train_mask]
X_train = X[train_mask]
Y_train = Y_onehot[train_mask]
A_test = A[test_mask][:, test_mask]
X_test = X[test_mask]
Y_test = Y_onehot[test_mask]

In [35]:
# Create model


In [36]:
model = GCNRW(n_features=X.size(1), n_classes=n_classes,  n_hidden=10000, lambda_=100)

In [37]:
# Train on training set

In [38]:
start = time.time()
Z_train = model.fit(A_train, X_train, Y_train)

In [39]:
# Evaluate on train set

In [40]:
Z_train = model.predict(A_train, X_train)
train_acc = accuracy_score(Y_train.argmax(1), Z_train.argmax(1))
print("train time",time.time() - start)
print(f'Train accuracy: {train_acc:.4f}')

train time 0.9165692329406738
Train accuracy: 0.9840


In [41]:
# Evaluate on test set

In [42]:
Z_test = model.predict(A_test, X_test)
test_acc = accuracy_score(Y_test.argmax(1), Z_test.argmax(1))
print(f'Test accuracy: {test_acc:.4f}')

Test accuracy: 0.7125


In [43]:
#Employing the code on PubMed data

In [44]:
# Load data

In [45]:
dataset = Planetoid(root='/tmp/PubMed', name='PubMed')
data = dataset[0]
A = data.edge_index
X = data.x
Y = data.y
n_classes = dataset.num_classes

Processing...
Done!


In [46]:
# Convert target to one-hot

In [47]:
Y_onehot = F.one_hot(Y, num_classes=n_classes).float()

In [48]:
# Get adjacency matrix

In [49]:
num_nodes = data.num_nodes
A = torch.zeros(num_nodes, num_nodes)
A[data.edge_index[0], data.edge_index[1]] = 1

In [50]:
# Split data into training and test sets

In [51]:
train_mask, test_mask = train_test_split(range(num_nodes), test_size=0.85, stratify=Y)

A_train = A[train_mask][:, train_mask]
X_train = X[train_mask]
Y_train = Y_onehot[train_mask]
A_test = A[test_mask][:, test_mask]
X_test = X[test_mask]
Y_test = Y_onehot[test_mask]

In [52]:
# Create model

In [54]:
model = GCNRW(n_features=X.size(1), n_classes=n_classes,  n_hidden=7000, lambda_=10)

In [ ]:
# Train on training set

In [55]:
start = time.time()
Z_train = model.fit(A_train, X_train, Y_train)

In [ ]:
# Evaluate on train set

In [56]:
Z_train = model.predict(A_train, X_train)
train_acc = accuracy_score(Y_train.argmax(1), Z_train.argmax(1))
print("train time",time.time() - start)
print(f'Train accuracy: {train_acc:.4f}')

train time 7.368612766265869
Train accuracy: 0.8965


In [ ]:
# Evaluate on test set

In [57]:
Z_test = model.predict(A_test, X_test)
test_acc = accuracy_score(Y_test.argmax(1), Z_test.argmax(1))

print(f'Test accuracy: {test_acc:.4f}')

Test accuracy: 0.8024
